<a href="https://colab.research.google.com/github/hasini-06/RAG-Assignment2/blob/main/RAG_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers pymupdf sentence-transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 15.0 MB/s eta 0

**Uploading Files**

In [ ]:
from google.colab import files
uploaded = files.upload()   #uploading files
pdf_path = list(uploaded.keys())[0]   #get the uploaded filename

Saving GEN_AI__Questions_and_Answers.pdf to GEN_AI__Questions_and_Answers.pdf


**Extracting Text from pdf**

In [ ]:
import fitz  # PyMuPDF

# Function to extract all text from the uploaded PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

document_text = extract_text_from_pdf(pdf_path)

**Chunking**

In [ ]:
import textwrap

# Function to split the long text into manageable chunks (default 300 characters)
def chunk_text(text, chunk_size=300):
    return textwrap.wrap(text, width=chunk_size)

chunks = chunk_text(document_text)  # List of text chunks
print(f"No. of chunks: {len(chunks)}")
print("Sample Chunk:", chunks[0])  # Displaying the first chunk

No. of chunks: 16
Sample Chunk:                                 Questions and Answers     1) What is LLM? Which platform contains LLM's?  A): LLM stands for Large Language Model. These are AI models  trained on huge amount of data to perform NLP tasks. Examples:  ChatGPT (OpenAI), Gemini (Google), Claude  (Anthropic), and models


**Embeddings**

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

# Load pre-trained sentence embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings for all chunks
embeddings = embed_model.encode(chunks).tolist()

# Initialize ChromaDB client (in-memory)
client = chromadb.Client(Settings(anonymized_telemetry=False))

# Create or get a collection named "pdf_chunks"
collection = client.get_or_create_collection(name="pdf_chunks")

# Add all chunks and their corresponding embeddings to the collection
for i, chunk in enumerate(chunks):
    collection.add(
        ids=[f"chunk_{i}"],              # Unique ID for each chunk
        documents=[chunk],              # Original chunk text
        embeddings=[embeddings[i]]      # Corresponding embedding
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


**Loading the Model**

In [ ]:
from transformers import pipeline

# Load a small FLAN-T5 model for generating answers based on retrieved context
generator = pipeline("text2text-generation", model="google/flan-t5-small")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


**Retrieving**

In [ ]:
# Function to handle the query: retrieve relevant chunks and generate an answer
def retrieve_and_answer(query, top_k=1):
    # Embed the query
    query_embedding = embed_model.encode([query])[0]

    # Search the most relevant chunks from ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    # Get the top-k retrieved chunk texts
    retrieved_texts = results['documents'][0]
    context = " ".join(retrieved_texts)  # Combine into a single string

    # Prepare prompt for the model
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"

    # Generate the answer using the FLAN-T5 model
    result = generator(prompt, max_length=100)
    return result[0]['generated_text']

**User Query and Ouput**

In [ ]:
question = input("Enter your query: ")
answer = retrieve_and_answer(question)

print("\nQue:", question)
print("Ans:", answer)

Enter your query: What is an LLM?


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Que: What is an LLM?
Ans: LLM stands for Large Language Model. These are AI models trained on huge amount of data to perform NLP tasks. Examples: ChatGPT (OpenAI), Gemini (Google), Claude (Anthropic), and models
